# `NHITS` with NeuralForecast

### Loading Libraries

In [1]:
%cd ../..

/Users/joaquinromero/Desktop


In [ ]:
# First, install the missing package
!pip install neuralforecast

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# OS 
import os
import shutil
import joblib

# Data Visualization
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

# JavaScript Object Notation
import json

# Path
from pathlib import Path
from tqdm.autonotebook import tqdm

# IPython & Itertools
from itertools import cycle
from IPython.display import display, HTML

# Stats Forecast
from statsforecast import StatsForecast

# NeuralForecast - will work after installation
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.auto import AutoNBEATS
from neuralforecast.losses.pytorch import MQLoss

from statsforecast import StatsForecast
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS
from neuralforecast.auto import AutoNHITS
from neuralforecast.losses.pytorch import MQLoss

# FuncTools
from functools import partial

In [ ]:
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import rmse, mae, mse, mase

In [ ]:
# %load_ext autoreload

# %autoreload 2

In [ ]:
tqdm.pandas()

np.random.seed(42)

pio.templates.default = "plotly_white"

In [ ]:
os.makedirs("imgs/chapter_16", exist_ok=True)

preprocessed = Path.home() / "Desktop" / "data" / "london_smart_meters" / "preprocessed"

output = Path.home() / "Desktop" / "data" / "london_smart_meters" / "output"

In [ ]:
TRAIN_SUBSAMPLE = False  # Trains a subsample of IDs to improve run speed

RETUNE = True  # if false, will use pre-trained hyperparameters when generating the AUTO NeuralForecast

In [ ]:
preprocessed = Path("data/london_smart_meters/preprocessed")
output = Path("data/london_smart_meters/output")

try:
    #Reading the missing value imputed and train test split data
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    # Read in the Validation dataset as test_df so that we predict on it
    test_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    # test_df = pd.read_parquet(preprocessed/"selected_blocks_test_missing_imputed_feature_engg.parquet")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

In [ ]:
train_df.head(2)

In [ ]:
print("Total # of IDs Pre-Sampling: ", len(train_df.LCLid.unique()))

In [ ]:
StatsForecast.plot(train_df,
                   engine='matplotlib',
                   id_col='LCLid',
                   time_col= 'timestamp',
                   target_col='energy_consumption')

In [ ]:
# To run on smaller set of data for daster iteration.
if TRAIN_SUBSAMPLE:
    print("sub sampling")
    SAMPLE = 10
    sampled_LCLids = pd.Series(train_df.LCLid.unique().remove_unused_categories().categories).sample(SAMPLE, random_state=99).tolist()
    train_df = train_df.loc[train_df.LCLid.isin(sampled_LCLids)]
    test_df = test_df.loc[test_df.LCLid.isin(sampled_LCLids)]

In [ ]:
print("Total # of IDs Post Sampling: ", len(train_df.LCLid.unique()))

### Train, Validation, Test Set

In [ ]:
print("Training Min Date: ", train_df.timestamp.min(), 
      "\nTraining Max Date: ", train_df.timestamp.max(), 
      "\nTesting Min Date: ", test_df.timestamp.min(),
      "\nTesting Max Date: ", test_df.timestamp.max()
)

In [ ]:
#Keeping 1 days aside as a validation set
cutoff = train_df.timestamp.max() - pd.Timedelta(1, "D")

validation_df = train_df[(train_df.timestamp>cutoff)].reset_index(drop=True) # validation prediction set
training_df = train_df[(train_df.timestamp<=cutoff)].reset_index(drop=True) # training set used for validation set

print(f"Train Max: {training_df.timestamp.max()} \nValidation Min: {validation_df.timestamp.min()} \nValidation Max: {validation_df.timestamp.max()}")
print(f"Validation Horizon: {len(validation_df.timestamp.unique())}")

In [ ]:
h = 48

max_steps = 100

### Training NHITS Model

In [ ]:
model_untuned = [NHITS(h=h,  input_size = 48*7,              
                max_steps=max_steps)]

model_untuned = NeuralForecast(models=model_untuned, freq='30min')
model_untuned.fit(training_df[['LCLid','timestamp','energy_consumption']], id_col = 'LCLid',time_col = 'timestamp',target_col='energy_consumption')


In [ ]:
# Get Predictions for validation
pred_df =  model_untuned.predict(futr_df=validation_df[['LCLid','timestamp','energy_consumption']]).reset_index()
pred_df = pred_df.merge(validation_df[['LCLid','timestamp','energy_consumption']], on=['LCLid','timestamp'], how='left')
pred_df.head()

In [ ]:
#visualize results
StatsForecast.plot(validation_df[['LCLid','timestamp','energy_consumption']], 
                   pred_df, engine='matplotlib', 
                   id_col='LCLid',
                   time_col= 'timestamp', 
                   target_col='energy_consumption',
                   models=['NHITS'])

#### Evaluate `NHITS` Forecast

In [ ]:
fcst_mase = partial(mase, seasonality=48)

# Get metrics for individual LCLid's
NHITS_metrics = evaluate(pred_df, 
        metrics=[rmse, mae, mse, fcst_mase],  
        train_df = train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )

# Get aggregated metrics for across all LCLid's by model
NHITS_metrics_agg = evaluate(pred_df, 
        metrics=[rmse, mae, mse, fcst_mase],  
        train_df = train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        agg_fn='mean'
        )

In [ ]:
NHITS_metrics_agg

### NHITS Tuned

In [ ]:
# Define the file path
config_file_path = 'notebooks/Chapter16/saved_params_config/NHITS_best_config.json'
try:
    with open(config_file_path, 'r') as config_file:
        loaded_config = json.load(config_file)
        print(loaded_config)
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. 
    </div>
    """))

In [ ]:
NHITS_config = {
    "max_steps": max_steps,  # This parameter can be adjusted if needed
    #"input_size": 48*7*2,  # Size of input window
    "input_size": tune.choice([h,h*7,h*7*2,h*7*3]),
    "learning_rate": tune.loguniform(1e-2, 1e-1),  # Initial learning rate
    "scaler_type": tune.choice(["minmax", "robust", "standard"]),
    #"dropout_prob_theta":tune.uniform(0, .1),
    "batch_size": tune.choice([32, 64])    
}

if RETUNE == True:
    models = [AutoNHITS(h=h, 
                     config = NHITS_config,
                     search_alg = HyperOptSearch(),
                     backend = 'ray',
                     num_samples = 100,
                     cpus=1)]

else:
    models = [AutoNHITS(h=h, 
                    config = loaded_config,
                    search_alg = None,
                    backend = 'ray',
                    cpus=1)]


model_tuned = NeuralForecast(models=models, freq='30min')
model_tuned.fit(training_df[['LCLid','timestamp','energy_consumption']], id_col = 'LCLid',time_col = 'timestamp',target_col='energy_consumption', verbose = True)


In [ ]:
if RETUNE == True:
    NHITS_best_config = model_tuned.models[0].results.get_best_result().config

    # Remove specific keys using the pop method and then saving so we can extract parameters later
    NHITS_best_config.pop("loss", None)
    NHITS_best_config.pop("valid_loss", None)
    NHITS_best_config.pop("h", None)

    # Save the filtered configuration to a JSON file
    with open(config_file_path, 'w') as config_file:
        json.dump(NHITS_best_config, config_file, indent=4)

    print("Best configuration as string:")
    print(NHITS_best_config)

In [ ]:
results = model_tuned.models[0].results.get_dataframe()

results[['loss', 'train_loss', 'timestamp', 
       'training_iteration', 
        'config/max_steps', 'config/input_size',
       'config/learning_rate', 'config/h', 'config/loss',
       ]].head(2)

In [ ]:
pred_df_autoNHITS =  model_tuned.predict(futr_df=validation_df[['LCLid','timestamp','energy_consumption']]).reset_index()
pred_df = pred_df_autoNHITS.merge(pred_df[['LCLid','timestamp','energy_consumption','NHITS']], on=['LCLid','timestamp'], how='left')
pred_df.head()

In [ ]:
fcst_mase = partial(mase, seasonality=48)

autoNHITS_metrics = evaluate(pred_df, 
        metrics=[rmse, mae, mse, fcst_mase],  
        train_df = train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption'
        )

autoNHITS_metrics_agg = evaluate(pred_df, 
        metrics=[rmse, mae, mse, fcst_mase],  
        train_df = train_df[['timestamp', 'LCLid', 'energy_consumption']],      
        id_col = 'LCLid',
        time_col = 'timestamp',
        target_col = 'energy_consumption',
        agg_fn='mean'
        )

In [ ]:
autoNHITS_metrics_agg

### Interpretability

In [ ]:
from neuralforecast.tsdataset import TimeSeriesDataset

model_interpretable = model_untuned.models[0]
dataset, *_ = TimeSeriesDataset.from_df(df = training_df[training_df.LCLid == training_df.LCLid[0]], id_col='LCLid',time_col='timestamp',target_col='energy_consumption')
y_hat = model_interpretable.decompose(dataset=dataset)

In [ ]:
validation_df.head(2)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(6, 7))

fig.suptitle(training_df.LCLid[0], fontsize=16)
ax[0].plot(validation_df[validation_df.LCLid == training_df.LCLid[0]]['energy_consumption'].values, label='True', linewidth=4)
ax[0].plot(y_hat.sum(axis=1).flatten(), label='Forecast', color="#7B3841")
ax[0].legend()
ax[0].set_ylabel('Harmonic Signal')

ax[1].plot(y_hat[0,2]+y_hat[0,0], label='stack1', color="green")
ax[1].set_ylabel('NHITS Trend Stack')

ax[2].plot(y_hat[0,1], label='stack2', color="orange")
ax[2].set_ylabel('NHITS Seasonality Stack')
ax[2].set_xlabel(r'Prediction $\tau \in \{t+1,..., t+H\}$')

plt.show()
fig.savefig('imgs/chapter_16/NHITS_interp.png')